# MODEL TRAINING  USING THE TF OBJECT DETECTION API

This notebook is based on modular [shell scripts](./scripts) for convenience. The scripts comprise minimal lines of code (LOC), on average, no more than 10 LOC for each, making them easy to read, learn, and adapt for future applications.

**!!!Caveat :** 

Some scripts can be conveniently executed directly from this notebook, and some need to be executed in terminal to grant permissions. I recommend running each script from the terminal to see STDOUT clearly. If run in terminal, use the [module-training](./) directory as the working directory.

***This is an academic project. Interested developers are invited to freely contribute issues, questions, improvements, and discussions.***

---

## 1. CHECK SUBMODULES

[`check-submodules.sh`](./scripts/) makes sure that the [LabelImg](https://github.com/tzutalin/labelImg) and [TF models](https://github.com/tensorflow/models) submodule repositories are initialized correctly.

In [12]:
!./scripts/check-submodules.sh

-- checking submodules


## 2. CREATE VIRTUAL ENVIRONMENT

[`venv.sh`](./scripts/) creates a virtual environment and installs essential [apt](./resources/apt.txt) and [pip](./resources/pip.txt) dependencies  [( see dependencies.sh )](./scripts/).Depending on your system, you may still need to install more dependencies in addition to yolov5's and labelImg's dependencies! The script should be executed from the terminal. This is necessary to grant install permissions. The default name of the virtual environment is **venv**. This name can, of course, be changed by simply passing a single argument representative of the name you want. From the working directory run 

`./scripts/venv.sh` # virtual environment name = venv

or 

`./scripts/venv.sh my-venv-name` # virtual environment name = my-venv-name

---

## 3. TEST SETUP USING [EXAMPLE IN YOLOv5 DOCS](https://github.com/ultralytics/yolov5/wiki/Train-Custom-Data)

In [ ]:
import uuid  
import os
import cv2
import time
import torch
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline 

# test with *s model (https://github.com/ultralytics/yolov5#pretrained-checkpoints)
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', force_reload=True); 
img = 'https://ultralytics.com/images/zidane.jpg'
results = model(img)
results.print()


plt.imshow(np.squeeze(results.render()))
plt.show()
#results.render()

## 4. Create [dataset.yaml spec](https://github.com/ultralytics/yolov5/wiki/Train-Custom-Data#1-create-datasetyaml) in the root directory of [Yolov5s' submodule directory](./external/yolov5) 

For this project, dataset.yaml looks like this

```
path: ../../resources/images
train: train
val: train

nc: 1  # number of classes
names: [ 'mobile device' ] # class  names

```

**!!!Caveat :** 

Also ensure to honor the location of the `labels` directory. It **must** exist in the same directory as the `images` directory (or whatever the directory's name where your training images are. The `labels` directory structure **must** also mirror the directory structure of the `images directory`!

---

## 4. TRAIN CUSTOM MODEL [Yolov5x model] (https://github.com/ultralytics/yolov5#pretrained-checkpoints)

Training speces for this project are as follows

* image size—320
* image batch—16
* image epochs—500
* image weights—yolov5x
* image workers—2

---

In [ ]:
!cd external/yolov5 && python3 train.py --img 320 --batch 16 --epochs 500 --data dataset.yaml --weights yolov5x.pt --workers 2

## 5. TEST TRAINED MODEL

---

In [ ]:
custom_model = torch.hub.load('ultralytics/yolov5', 'custom', path='external/yolov5/runs/train/exp7/weights/best.pt', force_reload=True)

img = 'resources/images/test/20_scene.png'
results = custom_model(img)
results.print()

plt.imshow(np.squeeze(results.render()))
plt.show()